# Creating Your First Aurora Cluster - Part 2

<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-Well-Build">What We'll Build</a></li>
<li><a href="#Prerequisite">Prerequisite</a></li>
<li><a href="#Cost-Overview">Cost Overview</a></li>
<li><a href="#Visual-Guide-to-Aurora-Console">Visual Guide to Aurora Console</a></li>
<li><a href="#Step-1-Create-IAM-Roles">Step 1: Create IAM Roles</a></li>
<li><a href="#Step-2-Create-DB-Subnet-Group">Step 2: Create DB Subnet Group</a></li>
<li><a href="#Step-3-Create-Parameter-Groups">Step 3: Create Parameter Groups</a></li>
<li><a href="#Step-4-Create-Aurora-Cluster">Step 4: Create Aurora Cluster</a></li>
<li><a href="#Step-5-Associate-S3-Import-Role">Step 5: Associate S3 Import Role</a></li>
<li><a href="#Step-6-Create-Writer-and-Reader-Instances">Step 6: Create Writer and Reader Instances</a></li>
<li><a href="#Step-7-Network-Diagnostics-and-Troubleshooting">Step 7: Network Diagnostics and Troubleshooting</a></li>
<li><a href="#Summary-and-Resources-Created">Summary and Resources Created</a></li>
<li><a href="#Cleanup">Cleanup</a></li>
<li><a href="#Next-Steps">Next Steps</a></li>
<li><a href="#Additional-Resources">Additional Resources</a></li>
</ul>
</div>

> **💡 Quick Start Available**: Want to skip the manual setup on **networking and database creation**? Launch the infrastructure with our [AWS CloudFormation template](../cfn/aurora-complete-stack.yml). If you have [set up networking already](2.1.1_create_your_first_aurora_postgresql_part1.ipynb), launch Aurora Serverless v2 with our [AWS CloudFormation template](https://console.aws.amazon.com/cloudformation/home/#stacks/create/template?stackName=AuroraServerlessV2&templateSource=Upload) - download the [template file](../cfn/aurora-serverless-v2.yml) first and upload it. Feel free to explore from the [AWS console](https://console.aws.amazon.com/rds) for what you have created. Then, go to [the next section to connect to your database](../2.2_Connecting_to_Your_Aurora_PostgreSQL/README.MD).

## What We'll Build

- [**✅ Network Setup**: VPC with public/private subnets across 2 AZs, routing, private subnet isolation, and security groups](./2.1.1_create_your_first_aurora_postgresql_part1.ipynb)
- **Database Layer**: Aurora PostgreSQL Serverless v2 cluster with writer and reader instances

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Networking Foundation**: Complete networking setup from either:
  - [SageMaker Notebook CloudFormation template](../../1_Getting_Started_with_AWS/cfn/sagemaker-notebook-template.yaml) (automated setup)
  - [Part 1: Create VPC and Networking](./2.1.1_create_your_first_aurora_postgresql_part1.ipynb) (manual setup)
- ✅ **AWS Account**: Active AWS account with appropriate permissions
  - [Create your first AWS account](../1_Getting_Started_with_AWS/README.MD) if you have not.
- ✅ **Jupyter Environment**: This notebook should be run in:
  - [Amazon SageMaker Jupyter notebook (recommended)](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
  - Or, local Jupyter with AWS CLI configured. Ensure [database access from your local Jupyter](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD).
- ✅ **IAM Permissions** for:
  - RDS cluster and instance creation/management
  - IAM role creation and policy attachment
  - Secrets Manager operations
  - Parameter group management
- ✅ **AWS CLI**: Configured with credentials and default region
  - [Amazon SageMaker Jupyter notebook (recommended)](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD) has an IAM role with permissions to access relevant resources. 

> 💡 **Note**: This notebook creates the Aurora PostgreSQL cluster. You'll connect to it in [Section 2.2](../2.2_Connecting_to_Your_Aurora_PostgreSQL/README.MD).

## Cost Overview 💰

Use [AWS Pricing Calculator](https://calculator.aws/#/) to estimate the cost for your architecture solution. The following a rough cost estimation for the resources created in this notebook.

| Component | Cost (us-east-1) | Notes |
|-----------|------------------|--------|
| Aurora Serverless v2 | \$0.12/ACU-hour | Minimum 0 ACUs with auto-pause |
| Storage | \$0.10/GB-month (standard) | Starts at 10GB, scales automatically |
| Backup | First day free | Keep retention at 1 day for free backups |
| Data API | \$0.35/million requests | HTTP endpoint for serverless applications |

💡 **Cost Optimization Tips:**
- Use auto-pause feature to allow [scaling down to 0 ACU](https://aws.amazon.com/blogs/database/introducing-scaling-to-0-capacity-with-amazon-aurora-serverless-v2/) for cost saving when database is idle
- Set appropriate min/max ACU values based on workload
- Leverage free tier features, such as [Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.Overview.html) with 7 days metrics retention period, 1 day [automatic backup retention](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Backups.Retaining.html) for Point-in-Time recovery.
- If the workload is not I/O intensive during testing phase, use standard storage instead of [I/O optimized](https://aws.amazon.com/blogs/aws/new-amazon-aurora-i-o-optimized-cluster-configuration-with-up-to-40-cost-savings-for-i-o-intensive-applications/).
- Data API is ideal for serverless applications with infrequent database access patterns

## Visual Guide to Aurora Console

### Creating Aurora Cluster
*Step-by-step console walkthrough of creating an Aurora cluster*

![Creating Aurora Cluster](../images/2.1-create-aurora-cluster-console.gif)

### Exploring Aurora Console
*Overview of key Aurora console features:*
- Configuration settings
- Chat with Amazon Q for best practice suggestions based on your particular settings

![Exploring Aurora Console](../images/2.1-explore-aurora-cluster-console.gif)

## Step-by-step Guidance

The following steps will create a complete Aurora PostgreSQL Serverless v2 cluster with production-ready features:

- **IAM Roles**: S3 import and enhanced monitoring capabilities
- **Network Configuration**: Database subnet groups and security settings
- **Security**: Secrets Manager integration for credential management
- **Database Setup**: Aurora cluster with custom parameter groups
- **Instances**: Writer and reader instances with Performance Insights
- **Diagnostics**: Network and cluster health verification

## Step 1: Create IAM Roles

Create essential IAM roles that Aurora will use for S3 data import operations. The S3 import role allows Aurora to read data from S3 buckets. You'll see the ARNs for the role created in the output.

In [ ]:
%%bash
# Create IAM role for S3 import
echo "Creating IAM role for S3 import..."
aws iam create-role \
    --role-name aurora-s3-import-role \
    --assume-role-policy-document '{
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "rds.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }' \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8

# Attach S3 read policy to the role
aws iam attach-role-policy \
    --role-name aurora-s3-import-role \
    --policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess

# Get the role ARN for later use
S3_ROLE_ARN=$(aws iam get-role --role-name aurora-s3-import-role --query 'Role.Arn' --output text)
echo "S3 Import Role ARN: $S3_ROLE_ARN"

# Create IAM role for enhanced monitoring
echo "Creating IAM role for enhanced monitoring..."
aws iam create-role \
    --role-name aurora-monitoring-role \
    --assume-role-policy-document '{
        "Version": "2012-10-17",
        "Statement": [{
            "Effect": "Allow",
            "Principal": {"Service": "monitoring.rds.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }]
    }' \
    --tags Key=CreationSource,Value=aws-database-cookbook-v2025.8

# Attach enhanced monitoring policy
aws iam attach-role-policy \
    --role-name aurora-monitoring-role \
    --policy-arn arn:aws:iam::aws:policy/service-role/AmazonRDSEnhancedMonitoringRole

# Get monitoring role ARN
MONITORING_ROLE_ARN=$(aws iam get-role --role-name aurora-monitoring-role --query 'Role.Arn' --output text)
echo "Monitoring Role ARN: $MONITORING_ROLE_ARN"

# Save for use in other cells
echo "export S3_ROLE_ARN=$S3_ROLE_ARN" > .aurora_vars
echo "export MONITORING_ROLE_ARN=$MONITORING_ROLE_ARN" >> .aurora_vars

## Step 2: Create DB Subnet Group

> **💡 Important**: Before proceeding, make sure you have the private subnet IDs available. Check the private subnets first to set these environmental variables.

### Detect Private Subnets

Run the cell below to detect and set the private subnet variables needed for the Aurora cluster.

The following script automatically detects your VPC configuration by checking for CloudFormation stacks and retrieving private subnet IDs needed for Aurora cluster creation. The commands search for existing cookbook or aurora stacks and extract subnet information from either CloudFormation outputs or direct VPC queries. You'll see the detected private subnet IDs that will be used for the Aurora database subnet group.

In [ ]:
%%bash
# First, check for SageMaker notebook CloudFormation stack
SAGEMAKER_STACK=$(aws cloudformation list-stacks --stack-status-filter CREATE_COMPLETE UPDATE_COMPLETE --query "StackSummaries[?contains(StackName, 'SagemakerJupyterNotebook') || contains(StackName, 'cookbook')].StackName" --output text | head -1)

if [ ! -z "$SAGEMAKER_STACK" ]; then
    echo "Found SageMaker notebook CloudFormation stack: $SAGEMAKER_STACK"
    # Get VPC ID from SageMaker stack
    VPC_ID=$(aws cloudformation describe-stacks --stack-name $SAGEMAKER_STACK --query "Stacks[0].Outputs[?OutputKey=='VpcId'].OutputValue" --output text)
    
    if [ ! -z "$VPC_ID" ] && [ "$VPC_ID" != "None" ]; then
        echo "Using SageMaker notebook VPC: $VPC_ID"
        # Get private subnets from the SageMaker VPC
        PRIVATE_SUBNET_A=$(aws cloudformation describe-stacks --stack-name $SAGEMAKER_STACK --query "Stacks[0].Outputs[?OutputKey=='PrivateSubnetA'].OutputValue" --output text)
        PRIVATE_SUBNET_B=$(aws cloudformation describe-stacks --stack-name $SAGEMAKER_STACK --query "Stacks[0].Outputs[?OutputKey=='PrivateSubnetB'].OutputValue" --output text)
        
        if [ ! -z "$PRIVATE_SUBNET_A" ] && [ ! -z "$PRIVATE_SUBNET_B" ]; then
            echo "Found private subnets from SageMaker stack:"
            echo "PRIVATE_SUBNET_A: $PRIVATE_SUBNET_A"
            echo "PRIVATE_SUBNET_B: $PRIVATE_SUBNET_B"
            
            # Get database security group from CloudFormation
            SG_ID=$(aws cloudformation describe-stacks --stack-name $SAGEMAKER_STACK --query "Stacks[0].Outputs[?OutputKey=='DatabaseSecurityGroup'].OutputValue" --output text)
            if [ ! -z "$SG_ID" ] && [ "$SG_ID" != "None" ]; then
                echo "Found database security group from CloudFormation: $SG_ID"
            fi
        else
            echo "Private subnets not found in CloudFormation stack"
        fi
    else
        echo "VPC not found in CloudFormation stack"
    fi
else
    echo "No SageMaker CloudFormation stack found"
fi

# If subnets not found from SageMaker stack, check for manually created VPC
if [ -z "$PRIVATE_SUBNET_A" ] || [ -z "$PRIVATE_SUBNET_B" ]; then
    echo "Looking for manually created subnets with aurora-private tags..."
    PRIVATE_SUBNET_A=$(aws ec2 describe-subnets --filters "Name=tag:Name,Values=aurora-private-1a" --query "Subnets[0].SubnetId" --output text)
    PRIVATE_SUBNET_B=$(aws ec2 describe-subnets --filters "Name=tag:Name,Values=aurora-private-1b" --query "Subnets[0].SubnetId" --output text)
    
    if [ ! -z "$PRIVATE_SUBNET_A" ] && [ "$PRIVATE_SUBNET_A" != "None" ] && [ ! -z "$PRIVATE_SUBNET_B" ] && [ "$PRIVATE_SUBNET_B" != "None" ]; then
        echo "Found manually created private subnets:"
        echo "PRIVATE_SUBNET_A: $PRIVATE_SUBNET_A"
        echo "PRIVATE_SUBNET_B: $PRIVATE_SUBNET_B"
        # Get VPC ID from the manually created subnets
        VPC_ID=$(aws ec2 describe-subnets --subnet-ids $PRIVATE_SUBNET_A --query 'Subnets[0].VpcId' --output text)
        
        # Create security group for manual setup
        if [ -z "$SG_ID" ] || [ "$SG_ID" == "None" ]; then
            echo "Creating security group for Aurora database..."
            SG_ID=$(aws ec2 create-security-group \
                --group-name aurora-sg \
                --description "Security group for Aurora cluster" \
                --vpc-id $VPC_ID \
                --tag-specifications 'ResourceType=security-group,Tags=[{Key=Name,Value=aurora-sg}]' \
                --query 'GroupId' \
                --output text)
            
            # Add inbound rule for PostgreSQL from VPC
            aws ec2 authorize-security-group-ingress \
                --group-id $SG_ID \
                --protocol tcp \
                --port 5432 \
                --cidr $(aws ec2 describe-vpcs --vpc-ids $VPC_ID --query 'Vpcs[0].CidrBlock' --output text)
            
            echo "Created security group: $SG_ID"
        fi
    else
        echo "Could not find private subnets. Please run Part 1 first or use CloudFormation template."
        exit 1
    fi
fi

# Save environment variables
echo "export PRIVATE_SUBNET_A=$PRIVATE_SUBNET_A" > .env_vars
echo "export PRIVATE_SUBNET_B=$PRIVATE_SUBNET_B" >> .env_vars
echo "export VPC_ID=$VPC_ID" >> .env_vars
echo "export SAGEMAKER_STACK=$SAGEMAKER_STACK" >> .env_vars
echo "export SG_ID=$SG_ID" >> .env_vars

echo "Using VPC: $VPC_ID"
echo "Environment variables saved to .env_vars"

### Create subnet groups

Create a database subnet group that specifies which private subnets Aurora can use for database instances, ensuring proper network isolation and multi-AZ deployment. The subnet group associates the previously created private subnets with descriptive tags for resource management. You'll see the subnet group creation confirmation with associated subnet details.

In [ ]:
%%bash
# Source environment variables
source .env_vars

# Generate unique suffix for subnet group
SUBNET_GROUP_SUFFIX=$(date +%s | tail -c 6)
SUBNET_GROUP_NAME="aurora-subnet-group-$SUBNET_GROUP_SUFFIX"

echo "Creating DB subnet group with name: $SUBNET_GROUP_NAME"
aws rds create-db-subnet-group \
    --db-subnet-group-name $SUBNET_GROUP_NAME \
    --db-subnet-group-description "Subnet group for Aurora cluster" \
    --subnet-ids "$PRIVATE_SUBNET_A" "$PRIVATE_SUBNET_B" \
    --tags Key=Environment,Value=Development

# Save subnet group name to .env_vars for later usage
echo "export SUBNET_GROUP_NAME=$SUBNET_GROUP_NAME" >> .env_vars

# Verify subnet group creation
aws rds describe-db-subnet-groups \
    --db-subnet-group-name $SUBNET_GROUP_NAME \
    --query 'DBSubnetGroups[0].{Name:DBSubnetGroupName,Status:SubnetGroupStatus,Subnets:Subnets[*].SubnetIdentifier}' \
    --output table

## Step 3: Create Parameter Groups

Create custom parameter groups for both Aurora cluster and database instance levels, allowing fine-tuned configuration of PostgreSQL settings and security parameters. The parameter groups enable SSL enforcement and performance monitoring extensions like pg_stat_statements. You'll see parameter group creation confirmations and configuration updates for both cluster and instance levels.

In [ ]:
%%bash
echo "Creating cluster parameter group..."
aws rds create-db-cluster-parameter-group \
    --db-cluster-parameter-group-name aurora-cluster-params \
    --db-parameter-group-family aurora-postgresql16 \
    --description "Custom cluster parameters for Aurora"

echo "Creating DB parameter group..."
aws rds create-db-parameter-group \
    --db-parameter-group-name aurora-instance-params \
    --db-parameter-group-family aurora-postgresql16 \
    --description "Custom instance parameters for Aurora"

# Modify key parameters
echo "Configuring parameters..."
aws rds modify-db-cluster-parameter-group \
    --db-cluster-parameter-group-name aurora-cluster-params \
    --parameters "ParameterName=rds.force_ssl,ParameterValue=0,ApplyMethod=pending-reboot" \
                 "ParameterName=shared_preload_libraries,ParameterValue=pg_stat_statements\,pg_hint_plan\,auto_explain,ApplyMethod=pending-reboot"

aws rds modify-db-parameter-group \
    --db-parameter-group-name aurora-instance-params \
    --parameters "ParameterName=log_rotation_age,ParameterValue=1440,ApplyMethod=pending-reboot" \
                 "ParameterName=log_rotation_size,ParameterValue=102400,ApplyMethod=pending-reboot"

## Step 4: Create Aurora Cluster

Create an Aurora PostgreSQL Serverless v2 cluster with managed secret for enhanced security. The cluster uses cost-optimized settings including auto-pause functionality, capacity scaling (0-4 ACU), and standard storage configuration. AWS automatically creates and manages the master user password in Secrets Manager. You'll see cluster creation progress and a wait period for availability confirmation.

In [ ]:
%%bash
# Source variables
source .env_vars

echo "Using security group: $SG_ID"

echo "Creating Aurora PostgreSQL cluster with managed secret..."
aws rds create-db-cluster \
    --db-cluster-identifier aurora-demo \
    --engine aurora-postgresql \
    --engine-version 16.8 \
    --master-username masteruser \
    --manage-master-user-password \
    --database-name mylab \
    --db-subnet-group-name $SUBNET_GROUP_NAME \
    --vpc-security-group-ids $SG_ID \
    --db-cluster-parameter-group-name aurora-cluster-params \
    --serverless-v2-scaling-configuration MinCapacity=0,MaxCapacity=4,SecondsUntilAutoPause=600 \
    --backup-retention-period 1 \
    --storage-encrypted \
    --enable-http-endpoint \
    --enable-iam-database-authentication \
    --tags Key=Environment,Value=Development \
    --no-deletion-protection \
    --engine-lifecycle-support open-source-rds-extended-support-disabled

echo "Waiting for cluster to become available (this may take a few minutes)..."
aws rds wait db-cluster-available --db-cluster-identifier aurora-demo

### Verify Cluster Status

Check that the Aurora cluster is up and running with all expected configurations before proceeding to the next steps.

In [ ]:
%%bash
echo "🔍 Verifying Aurora cluster status..."

# Check cluster status and configuration
CLUSTER_STATUS=$(aws rds describe-db-clusters --db-cluster-identifier aurora-demo --query 'DBClusters[0].Status' --output text)

if [ "$CLUSTER_STATUS" = "available" ]; then
    echo "✅ Cluster is available and ready"
    
    # Display cluster details
    aws rds describe-db-clusters --db-cluster-identifier aurora-demo \
        --query 'DBClusters[0].{Status:Status,Engine:Engine,Version:EngineVersion,Endpoint:Endpoint,ReaderEndpoint:ReaderEndpoint,ServerlessV2:ServerlessV2ScalingConfiguration}' \
        --output table
    
    echo "✅ Cluster verification complete - ready for next steps"
else
    echo "❌ Cluster status: $CLUSTER_STATUS"
    echo "Please wait for cluster to become available before proceeding"
    exit 1
fi

## Step 5: Associate S3 Import Role with Cluster

Associate the previously created S3 import IAM role with the Aurora cluster, enabling the aws_s3 extension to import and export data directly from S3 buckets. The role attachment provides the necessary permissions for Aurora to access S3 resources securely. You'll see confirmation of the S3 import capability being enabled for the cluster.

In [ ]:
%%bash
# Source variables
source .aurora_vars
source .env_vars

echo "Adding S3 import role to cluster..."
aws rds add-role-to-db-cluster \
    --db-cluster-identifier aurora-demo \
    --role-arn $S3_ROLE_ARN \
    --feature-name s3Import

echo "✅ S3 import capability enabled"

## Step 6: Create Writer and Reader Instances

Create both writer and reader instances for the Aurora cluster using the db.serverless instance class with Performance Insights enabled for monitoring. Both instances use custom parameter groups and include 7-day Performance Insights retention within the free tier. You'll see instance creation progress, availability confirmation, and detailed instance information including status and availability zone for both instances.

In [ ]:
%%bash
# Source variables
source .aurora_vars
source .env_vars

echo "Creating primary writer instance..."
aws rds create-db-instance \
    --db-instance-identifier aurora-writer \
    --db-cluster-identifier aurora-demo \
    --db-instance-class db.serverless \
    --engine aurora-postgresql \
    --db-parameter-group-name aurora-instance-params \
    --enable-performance-insights \
    --performance-insights-retention-period 7 \
    --monitoring-interval 60 \
    --monitoring-role-arn $MONITORING_ROLE_ARN \
    --tags Key=Role,Value=Writer

echo "Waiting for writer instance to become available (this may take 5-10 minutes)..."
aws rds wait db-instance-available --db-instance-identifier aurora-writer

echo "Creating reader instance..."
aws rds create-db-instance \
    --db-instance-identifier aurora-reader \
    --db-cluster-identifier aurora-demo \
    --db-instance-class db.serverless \
    --engine aurora-postgresql \
    --db-parameter-group-name aurora-instance-params \
    --enable-performance-insights \
    --performance-insights-retention-period 7 \
    --monitoring-interval 60 \
    --monitoring-role-arn $MONITORING_ROLE_ARN \
    --tags Key=Role,Value=Reader

echo "Waiting for reader instance to become available (this may take 5-10 minutes)..."
aws rds wait db-instance-available --db-instance-identifier aurora-reader

echo "✅ Instance details:"
aws rds describe-db-instances \
    --db-instance-identifier aurora-writer \
    --query 'DBInstances[0].{Instance:DBInstanceIdentifier,Status:DBInstanceStatus,Class:DBInstanceClass,AZ:AvailabilityZone}' \
    --output table

aws rds describe-db-instances \
    --db-instance-identifier aurora-reader \
    --query 'DBInstances[0].{Instance:DBInstanceIdentifier,Status:DBInstanceStatus,Class:DBInstanceClass,AZ:AvailabilityZone}' \
    --output table

### Production Recommendation: Add Reader Instance

> **💡 Important for Production**: For production environments, we strongly recommend creating at least one reader instance for:
> - High availability with automatic failover
> - Read scaling for reporting and analytics
> - Zero-downtime maintenance operations
>
> To add a reader instance, use the same command as above but with:
> - Different instance identifier (`aurora-reader`)
> - Different tag (`Role=Reader`)

## Step 7: Network Diagnostics and Troubleshooting

Let's run some network diagnostics to verify our setup and troubleshoot any connectivity issues. The diagnostics provide detailed information about network configuration, routing paths, and access controls for troubleshooting. You'll see formatted tables with security rules, routing information, and network topology details.

In [ ]:
%%bash
# Source environment variables
source .env_vars

echo "🔍 Running Network Diagnostics..."

echo "1. Checking Security Group Rules:"
aws ec2 describe-security-group-rules \
    --filters Name=group-id,Values=$SG_ID \
    --query 'SecurityGroupRules[?IpProtocol==`tcp` && FromPort==`5432`]' \
    --output table

echo "2. Checking Network ACLs:"
aws ec2 describe-network-acls \
    --filters Name=vpc-id,Values=$VPC_ID \
    --query 'NetworkAcls[*].{AclId:NetworkAclId,VpcId:VpcId,Entries:Entries[0:5]}' \
    --output table

echo "3. Checking Route Tables:"
aws ec2 describe-route-tables \
    --filters Name=vpc-id,Values=$VPC_ID \
    --query 'RouteTables[*].{RouteTableId:RouteTableId,Routes:Routes[*].{Destination:DestinationCidrBlock,Target:GatewayId||NatGatewayId||VpcPeeringConnectionId||TransitGatewayId||LocalGatewayId||CarrierGatewayId||NetworkInterfaceId||VpcEndpointId}}' \
    --output table

echo "4. Checking Subnet Connectivity:"
aws ec2 describe-subnets \
    --filters Name=vpc-id,Values=$VPC_ID \
    --query 'Subnets[*].{SubnetId:SubnetId,CIDR:CidrBlock,AZ:AvailabilityZone,Public:MapPublicIpOnLaunch}' \
    --output table

echo "✅ Network diagnostics complete!"

Check the Aurora cluster is healthy and available.

In [ ]:
%%bash
echo "🔍 Running Aurora Diagnostics..."

echo "1. Checking cluster status:"
aws rds describe-db-clusters \
    --db-cluster-identifier aurora-demo \
    --query 'DBClusters[0].{Status:Status,Engine:Engine,Version:EngineVersion}' \
    --output table

echo "2. Checking recent events:"
aws rds describe-events \
    --source-identifier aurora-demo \
    --source-type db-cluster \
    --duration 60 \
    --query 'Events[*].{Time:Date,Message:Message}' \
    --output table

echo "3. Checking Performance Insights enabled:"
aws rds describe-db-instances \
    --db-instance-identifier aurora-writer \
    --query 'DBInstances[0].PerformanceInsightsEnabled'

echo "✅ Aurora diagnostics complete!"

## Summary and Resources Created

Let's review what we've created in this lab. This summary script provides an overview of all AWS resources created during the Aurora cluster setup process. The output includes IAM roles, database resources, security components, and advanced features with their current status. You'll see a complete inventory of created resources and their configurations for reference and validation.

In [ ]:
%%bash
# Source variables
source .aurora_vars 2>/dev/null || true
source .env_vars 2>/dev/null || true

echo "📋 Resources Created Summary"

echo "1. IAM Roles:"
if [ ! -z "$S3_ROLE_ARN" ]; then
    echo "S3 Import Role: $S3_ROLE_ARN"
fi
if [ ! -z "$MONITORING_ROLE_ARN" ]; then
    echo "Monitoring Role: $MONITORING_ROLE_ARN"
fi

echo "2. Database Resources:"
aws rds describe-db-clusters --db-cluster-identifier aurora-demo --query 'DBClusters[0].{Cluster:DBClusterIdentifier,Status:Status,Engine:Engine,Version:EngineVersion,Endpoint:Endpoint}' --output table
aws rds describe-db-instances --query 'DBInstances[?DBClusterIdentifier==`aurora-demo`].{Instance:DBInstanceIdentifier,Status:DBInstanceStatus,Class:DBInstanceClass,Role:ReadReplicaSourceDBInstanceIdentifier||`Writer`}' --output table
if [ ! -z "$SUBNET_GROUP_NAME" ]; then
    aws rds describe-db-subnet-groups --db-subnet-group-name $SUBNET_GROUP_NAME --query 'DBSubnetGroups[0].{Name:DBSubnetGroupName,Status:SubnetGroupStatus}' --output table
fi

echo "3. Security Resources:"
# Get managed secret created by Aurora
MANAGED_SECRET=$(aws rds describe-db-clusters --db-cluster-identifier aurora-demo --query 'DBClusters[0].MasterUserSecret.SecretArn' --output text 2>/dev/null)
if [ "$MANAGED_SECRET" != "None" ] && [ ! -z "$MANAGED_SECRET" ]; then
    echo "Managed Secret (Aurora-created):"
    aws secretsmanager describe-secret --secret-id $MANAGED_SECRET --query '{Name:Name,Arn:ARN,Description:Description}' --output table
else
    echo "No managed secret found"
fi
if [ ! -z "$SG_ID" ]; then
    aws ec2 describe-security-groups --group-ids $SG_ID --query 'SecurityGroups[0].{GroupId:GroupId,Name:GroupName,VpcId:VpcId}' --output table
fi

echo "✅ Congratulations! You've successfully created a cost-optimized Aurora PostgreSQL cluster with managed secret."
echo "   Your database is now ready to use for development and testing purposes."

## Next Steps 🚀

Now that you've successfully created your Aurora PostgreSQL cluster with advanced features, it's time to learn how to connect to it and work with your data:

1. **Connect to Your Aurora Database**
   - Learn different connection methods in [2.2.1 Basic Connectivity](../2.2_Connecting_to_Your_Aurora_PostgreSQL/2.2.1_Basic_Connectivity.ipynb)
   - Understand connection pooling and security best practices

2. **Work with Your Data**
   - Create tables and load data in [2.2.2 Working with Data](../2.2_Connecting_to_Your_Aurora_PostgreSQL/2.2.2_Working_with_Data.ipynb)
   - Learn how to optimize queries and manage your database

3. **Implement Advanced Connection Management**
   - Set up IAM authentication and RDS Proxy in [2.2.3 Advanced Connection Management](../2.2_Connecting_to_Your_Aurora_PostgreSQL/2.2.3_Advanced_Connection_Management.ipynb)
   - Configure secure and efficient database access

Let's continue to [Section 2.2: Connecting to Your Aurora PostgreSQL](../2.2_Connecting_to_Your_Aurora_PostgreSQL/README.MD) to learn how to interact with your newly created database!

## Cleanup

> ⚠️ **Important**: The following commands will delete Aurora cluster created in this workshop. You will need an Aurora cluster for [4. Operational Excellence: Best Practices for Aurora](../../4_Operational_Excellence_Best_Practices_for_Aurora/README.md), [5. Scaling Your Aurora Database](../../5_Scaling_for_Success_Growing_with_Aurora/README.md), and [7. Break Free from Everything in One Database Trap: A Journey to Purpose-Built AWS Databases](../../7_Break_Free_from_Everything_in_One_Database_Trap_A_Journey_to_Purpose_Built_AWS_Databases/README.md). Only run these if you want to completely remove the Aurora cluster and associated resources. 

### Cost Consideration
Running these cleanup commands will help you avoid ongoing charges for:
- Aurora Serverless v2 compute (ACU hours)
- Aurora storage
- RDS Proxy
- Enhanced Monitoring

> 💡 **Note**: To clean up the Aurora cluster launched by the AWS CloudFormation template, then go to the [AWS CloudFormation console](https://console.aws.amazon.com/cloudformation/home/), select the AWS CloudFormation stack, and delete it.

Resources must be deleted in the correct order due to dependencies:

In [ ]:
%%bash
echo "🗑️ Starting cleanup process..."

# 1. Delete RDS Proxy
echo "Deleting RDS Proxy..."
aws rds delete-db-proxy --db-proxy-name aurora-demo-proxy 2>/dev/null || echo "RDS Proxy not found or already deleted"

# 2. Delete DB Instances
echo "Deleting DB instances..."
aws rds delete-db-instance --db-instance-identifier aurora-reader --skip-final-snapshot 2>/dev/null || echo "Reader instance not found"
aws rds delete-db-instance --db-instance-identifier aurora-writer --skip-final-snapshot 2>/dev/null || echo "Writer instance not found"

# Wait for instances to be deleted
echo "Waiting for instances to be deleted..."
sleep 30

# 3. Delete Aurora Cluster
echo "Deleting Aurora cluster..."
aws rds delete-db-cluster --db-cluster-identifier aurora-demo --skip-final-snapshot 2>/dev/null || echo "Cluster not found"

# 4. Delete DB Subnet Group
echo "Deleting DB subnet group..."
source .env_vars 2>/dev/null || true
aws rds delete-db-subnet-group --db-subnet-group-name ${SUBNET_GROUP_NAME:-aurora-subnet-group} 2>/dev/null || echo "Subnet group not found"

# 5. Delete Parameter Groups
echo "Deleting parameter groups..."
aws rds delete-db-cluster-parameter-group --db-cluster-parameter-group-name aurora-cluster-params 2>/dev/null || echo "Cluster parameter group not found"
aws rds delete-db-parameter-group --db-parameter-group-name aurora-instance-params 2>/dev/null || echo "Instance parameter group not found"

# 6. Delete Secret
echo "Deleting secret..."
aws secretsmanager delete-secret --secret-id aurora-demo-credentials --force-delete-without-recovery 2>/dev/null || echo "Secret not found"

# 7. Delete IAM Roles
echo "Deleting IAM roles..."
aws iam detach-role-policy --role-name aurora-s3-import-role --policy-arn arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess 2>/dev/null
aws iam delete-role --role-name aurora-s3-import-role 2>/dev/null || echo "S3 import role not found"

ACCOUNT_ID=$(aws sts get-caller-identity --query 'Account' --output text)
aws iam detach-role-policy --role-name aurora-proxy-role --policy-arn arn:aws:iam::$ACCOUNT_ID:policy/aurora-secrets-policy 2>/dev/null
aws iam delete-role --role-name aurora-proxy-role 2>/dev/null || echo "Proxy role not found"
aws iam delete-policy --policy-arn arn:aws:iam::$ACCOUNT_ID:policy/aurora-secrets-policy 2>/dev/null || echo "Secrets policy not found"

aws iam detach-role-policy --role-name aurora-monitoring-role --policy-arn arn:aws:iam::aws:policy/service-role/AmazonRDSEnhancedMonitoringRole 2>/dev/null
aws iam delete-role --role-name aurora-monitoring-role 2>/dev/null || echo "Monitoring role not found"

echo "✅ Cleanup completed. All Aurora resources have been deleted."

This step removes temporary files created during the workshop. 

In [ ]:
%%bash
# Cleanup temporary files
rm -f .env_vars .aurora_vars
echo "✅ Temporary files cleaned up"

## Additional Resources 📚

### Aurora Features
- [Aurora Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)
- [Performance Insights Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)
- [Serverless v2 Capacity Management](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/aurora-serverless-v2.html)

### Security & Authentication
- [IAM Database Authentication](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/UsingWithRDS.IAMDBAuth.html)
- [AWS Secrets Manager Integration](https://docs.aws.amazon.com/secretsmanager/latest/userguide/intro.html)
- [RDS Proxy Documentation](https://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/rds-proxy.html)